# 1. 타겟종이 존재하는 위치 주변의 정보 파악하기  

In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from haversine import haversine

## 1-1. 타겟종 주변의 무당벌레 종 분포 파악

In [3]:
# 무당벌레 전체 관측목록
df = pd.read_csv("../Data/20220209_07_21_무당벌레관측.csv")



# gps 오차 1000m미만만 남기기
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)


#Latitude, Longitude 0으로 기록된 부분들 삭제
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)


#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df['gps'] = df_gps   

#sp.로 들어간 애들을 제거
sp_idx = []
for i in range(len(df)):
    if 'sp.' in df.Species[i]:
        sp_idx.append(i)
df= df.drop(sp_idx).reset_index(drop=True)

# value_counts 해서 통계적으로 유의미한 30개도 안되는 애들 고르기
under_30 = []
for i in range(len(df.Species.value_counts())):
    if df.Species.value_counts()[i]<30:
        under_30.append(df.Species.value_counts().index[i])
        
        
# 고른애들과 이름이 같다면 drop해야함
drop_30 = []
for i in tqdm(range(len(df.Species))):
    for j in under_30:
        if df.Species[i] == j:
            drop_30.append(i)        
df= df.drop(drop_30).reset_index(drop=True)            
            
# 속만 있는 경우 제거
one_letter = []
for i in df.Species.value_counts().index:
    if len(i.split(' ')) == 1:
        one_letter.append(i)    
        
drop_one = []
for i in tqdm(range(len(df.Species))):
    for j in one_letter:
        if df.Species[i] == j:
            drop_one.append(i)        
df = df.drop(drop_one).reset_index(drop=True)

100%|███████████████████████████████████████████████████████████████████████| 175834/175834 [00:11<00:00, 14702.11it/s]


In [4]:
kellog = df[df.Species=='Brachiacantha ursina'].reset_index(drop=True)

In [6]:
kellog=kellog[(kellog.State!='Nuevo León')&(kellog.State!='Sonora')&(kellog.State!='Chihuahua')&(kellog.State!='Puebla')&
      (kellog.State!='Francisco Morazán')&(kellog.State!='Coahuila')&(kellog.State!='Chihuahua')&(kellog.State!='Durango')&
       (kellog.State!='San Luis Potosí')&(kellog.State!='Nayarit')&(kellog.State!='Veracruz')&(kellog.State!='México')&(kellog.State!='Jalisco')
      &(kellog.State!='Baja California')&(kellog.State!='District of Columbia')&(kellog.State!='Tamaulipas')
       &(kellog.State!='Michoacán')&(kellog.State!='Tlaxcala')&(kellog.State!='Oaxaca')&(kellog.State!='Morelos')
      &(kellog.State!='Guatemala')&(kellog.State!='Distrito Federal')&(kellog.State!='Hidalgo')        
      &(kellog.State!='Guanajuato')&(kellog.State!='Querétaro')     ].reset_index(drop=True)

In [7]:
kellog

,Unnamed: 0,Unnamed: 0.1,Source,Year,Species,Latitude,Longitude,public_positional_accuracy,State,gps
0,44,44,Bugguide.net,2007,Brachiacantha ursina,42.459085,-71.275566,0.0,Massachusetts,"(42.459085, -71.275566)"
1,129,129,"Museum of Comparative Zoology, Harvard University",2007,Brachiacantha ursina,42.317223,-71.008614,0.0,Massachusetts,"(42.31722259521485, -71.00861358642578)"
2,198,198,I-naturalist,2007,Brachiacantha ursina,43.076703,-89.521055,15.0,Wisconsin,"(43.0767029177, -89.5210551295)"
3,236,236,"Harvard University, Museum of Comparative Zoology",2007,Brachiacantha ursina,42.317222,-71.008611,0.0,Massachusetts,"(42.317222, -71.008611)"
4,423,423,Bugguide.net,2007,Brachiacantha ursina,42.028619,-94.855710,0.0,Iowa,"(42.028619, -94.85571)"
...,...,...,...,...,...,...,...,...,...,...
764,189445,189445,I-naturalist,2021,Brachiacantha ursina,46.609631,-72.702751,101.0,NaN,"(46.6096305847, -72.7027511597)"
765,189511,189511,I-naturalist,2021,Brachiacantha ursina,45.481655,-71.815905,14.0,NaN,"(45.4816548963, -71.8159053102)"
766,189940,189940,I-naturalist,2021,Brachiacantha ursina,45.427376,-75.764077,0.0,NaN,"(45.4273762997, -75.7640765997)"
767,189983,189983,I-naturalist,2021,Brachiacantha ursina,45.026950,-74.712937,4.0,NaN,"(45.02695, -74.7129366667)"


In [8]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in df.Species.value_counts().index:

    kellog[i+'_18'] = 0

In [9]:
# 주변 종 체크
for i in tqdm(range(len(kellog))): # target만 뽑아놓은 데이터 프레임
    for j in range(len(df.gps)):  # 전체 무당벌레 데이터 프레임
        if haversine(kellog.gps[i], df.gps[j], unit='km') <= 18 and kellog.Year[i] == df.Year[j]:
            kellog[df.Species[j]+'_18'][i]+=1

  0%|                                                                                          | 0/769 [00:00<?, ?it/s]<ipython-input-9-a2d82a62ea73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[df.Species[j]+'_18'][i]+=1
100%|████████████████████████████████████████████████████████████████████████████████| 769/769 [39:04<00:00,  3.05s/it]


In [10]:
kellog.iloc[:,10:].describe()

,Harmonia axyridis_18,Coccinella septempunctata_18,Hippodamia convergens_18,Coelophora maculata_18,Cycloneda sanguinea_18,Propylea quatuordecimpunctata_18,Cycloneda munda_18,Psyllobora vigintimaculata_18,Chilocorus stigma_18,Olla v-nigrum_18,...,Nephus flavifrons_18,Brachiacantha quadripunctata_18,Hyperaspis proba_18,Egius platycephalus_18,Hyperaspis connectens_18,Nephus intrusus_18,Exochomus fasciatus_18,Scymnus louisianae_18,Coccinella undecimpunctata_18,Delphastus pusillus_18
count,769.000000,769.000000,769.000000,769.000000,769.000000,769.000000,769.000000,769.000000,769.000000,769.000000,...,769.000000,769.000000,769.000000,769.0,769.000000,769.0,769.0,769.0,769.0,769.000000
mean,32.698309,5.414824,0.379714,3.882965,0.061118,3.195059,1.449935,1.065020,1.685306,0.033810,...,0.011704,0.014304,0.013004,0.0,0.001300,0.0,0.0,0.0,0.0,0.007802
std,66.230874,12.919556,2.339405,5.810088,0.830825,6.861800,3.712016,3.067545,3.436902,0.698884,...,0.148318,0.139019,0.113365,0.0,0.036061,0.0,0.0,0.0,0.0,0.088043
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
50%,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
75%,29.000000,4.000000,0.000000,5.000000,0.000000,3.000000,1.000000,1.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
max,667.000000,86.000000,37.000000,39.000000,20.000000,78.000000,34.000000,26.000000,34.000000,19.000000,...,2.000000,2.000000,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,1.000000


### 1-2. 타겟종 주변의 새 정보 파악

In [11]:
os.chdir(path = '../Data/E_bird/')

In [12]:
# Ebird에서 가져온 새 목록 합치기


# directory 내의 csv들을 전부 가져와서
for i in range(len(os.listdir())):
    globals()['bird{}'.format(i)] = pd.read_csv(os.listdir()[i],sep='\t')

# for_merge 라는 변수에 모두 합쳐놓음
for_merge = bird1.iloc[0:0]
for i in range(26):
    for_merge = pd.concat([for_merge,globals()['bird{}'.format(i)]])
bird2021 = pd.read_csv("../새2021.csv")
for_merge = pd.concat([for_merge, bird2021]).reset_index(drop=True)


# EBIRD자료는 GPS정확도를 제공하지 않으므로, GPS정확도가 존재한다면 INAT의 자료임
for_merge = for_merge[for_merge.coordinateUncertaintyInMeters.isnull()].reset_index(drop=True)

# 필요한 컬럼들만 남김
for_merge=for_merge[['species','year','month','decimalLatitude','decimalLongitude']].reset_index(drop=True)

# 종 이름이 NAN이면 쓸수 없으므로 DROP
for_merge=for_merge.drop(for_merge[for_merge.species.isnull()==True].index).reset_index(drop=True)

# haversine은 set에 들어있는 gps값을 받음 (lat, lon) 이형식으로 변환하는 과정
for_gps = []
for i in tqdm(range(len(for_merge))):
    for_gps.append( (for_merge['decimalLatitude'][i], for_merge['decimalLongitude'][i]) )
    
for_merge['gps'] = for_gps



for_merge=for_merge[(for_merge.species!='Locustella certhiola')&(for_merge.species!='Locustella ochotensis')&
         (for_merge.species!='Ianthocincla pectoralis')&(for_merge.species!='Luscinia calliope')&
         (for_merge.species!='Iduna aedon')&(for_merge.species!='Phylloscopus sibillatrix')&
         (for_merge.species!='Muscicapa latirostris')&(for_merge.species!='Tiaris bicolor')&
         (for_merge.species!='Tiaris canorus')&(for_merge.species!='Lonchura oryzivora')&
         (for_merge.species!='Lonchura cucullata')&(for_merge.species!='Estrilda coerulescens')&
         (for_merge.species!='alcarius ornatus')]

# 무당벌레는 관측지역에 잡힌 숫자를 믿을 수 없어서 관측 자체만 믿었음, 새 역시도 drop_duplicate로 동일한 절차 진행
for_merge = for_merge.drop_duplicates(keep='first').reset_index(drop=True)


# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,16,17,36,37,38,39,41,43,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|████████████████████████████████████████████████████████████████████| 3034469/3034469 [00:26<00:00, 116292.21it/s]


In [13]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

In [14]:
for i in tqdm(range(len(kellog))): # 타겟종만 뽑아놓은 데이터 프레임
    for j in range(len(for_merge.gps)):  # 전체 새 데이터 프레임
            if haversine(kellog.gps[i], for_merge.gps[j], unit='km') <= 18 and kellog.Year[i] == for_merge.year[j]:
                # 타겟종 관측 좌표와 새 관측좌표의 거리가 1km이하이고, 해당 좌표의 관측 연도가 같다면 count +1
                kellog[for_merge.species[j]+'_18'][i]+=1


  0%|                                                                                          | 0/769 [00:00<?, ?it/s]<ipython-input-14-5a383200347b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[for_merge.species[j]+'_18'][i]+=1
100%|██████████████████████████████████████████████████████████████████████████████| 769/769 [5:37:35<00:00, 26.34s/it]


In [15]:
kellog.to_csv("./ursina_조사.csv")

In [23]:
kellog.describe().iloc[:,100:]

,Lanius borealis_18,Calcarius lapponicus_18,Cinclus mexicanus_18,Lonchura punctulata_18,Zosterops japonicus_18,Centronyx henslowii_18,Paroaria coronata_18,Passer montanus_18,Calcarius ornatus_18,Pycnonotus cafer_18,...,Locustella lanceolata_18,Cyanerpes cyaneus_18,Acrocephalus schoenobaenus_18,Acrocephalus dumetorum_18,Taeniopygia guttata_18,Oenanthe pleschanka_18,Luscinia cyane_18,Phoenicurus phoenicurus_18,Pachyramphus major_18,Locustella fluviatilis_18
count,1090.000000,1090.000000,1090.000000,1090.000000,1090.0,1090.000000,1090.0,1090.000000,1090.000000,1090.0,...,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0
mean,6.051376,1.564220,2.319266,1.585321,0.0,0.056881,0.0,0.023853,0.061468,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,18.569580,3.686772,7.096275,14.042065,0.0,0.584835,0.0,0.569648,0.398438,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,4.000000,2.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,196.000000,59.000000,93.000000,313.000000,0.0,10.000000,0.0,17.000000,5.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
kellog

,Unnamed: 0,Unnamed: 0.1,Source,Year,Species,Latitude,Longitude,public_positional_accuracy,State,gps,...,Locustella lanceolata_18,Cyanerpes cyaneus_18,Acrocephalus schoenobaenus_18,Acrocephalus dumetorum_18,Taeniopygia guttata_18,Oenanthe pleschanka_18,Luscinia cyane_18,Phoenicurus phoenicurus_18,Pachyramphus major_18,Locustella fluviatilis_18
0,501,501,The Lost Ladybugs,2007,Coccinella trifasciata,49.246230,-124.778610,0.0,British Columbia,"(49.24623, -124.77861)",...,0,0,0,0,0,0,0,0,0,0
1,581,581,I-naturalist,2007,Coccinella trifasciata,37.239249,-122.071810,2.0,California,"(37.2392489426, -122.071810195)",...,0,0,0,0,0,0,0,0,0,0
2,616,616,I-naturalist,2007,Coccinella trifasciata,46.090159,-83.630013,488.0,Michigan,"(46.0901594073, -83.6300133176)",...,0,0,0,0,0,0,0,0,0,0
3,681,681,Centre for Biodiversity Genomics,2008,Coccinella trifasciata,51.140000,-115.420000,0.0,Alberta,"(51.14, -115.42)",...,0,0,0,0,0,0,0,0,0,0
4,695,695,NaN,2008,Coccinella trifasciata,51.148000,-115.420000,0.0,Alberta,"(51.148, -115.42)",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,190348,190348,I-naturalist,2021,Coccinella trifasciata,37.399112,-121.797335,0.0,California,"(37.3991116667, -121.797335)",...,0,0,0,0,0,0,0,0,0,0
1086,190420,37,LLP,2021,Coccinella trifasciata,47.596944,-92.501944,0.0,Minnesota,"(47.596944, -92.501944)",...,0,0,0,0,0,0,0,0,0,0
1087,191073,691,INAT,2021,Coccinella trifasciata,50.559764,-99.830498,8.0,NaN,"(50.5597644512, -99.8304984164)",...,0,0,0,0,0,0,0,0,0,0
1088,193628,3246,INAT,2021,Coccinella trifasciata,38.435761,-122.711401,9.0,California,"(38.4357609151, -122.7114008055)",...,0,0,0,0,0,0,0,0,0,0
